In [1]:
import os
import sys
import math
import glob
import shutil
import random
import subprocess
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

In [2]:
try:
    import ultralytics
except Exception:
    print("Installing ultralytics...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics==8.*"])
    import ultralytics

from ultralytics import YOLO

Installing ultralytics...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ydata-profiling 4.17.0 requires numpy<2.2,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
dataset_root = Path("/kaggle/input/the-3lc-cotton-weed-detection-challenge/cotton_weed_competition_dataset")
if not dataset_root.exists():
    # fallback: scan /kaggle/input
    for p in Path("/kaggle/input").iterdir():
        if "cotton" in p.name.lower() and "weed" in p.name.lower():
            dataset_root = p
            break
if not dataset_root.exists():
    raise FileNotFoundError("Dataset not found.")

In [4]:
train_images_dir = dataset_root / "train" / "images"
orig_train_labels_dir = dataset_root / "train" / "labels"
val_images_dir = dataset_root / "val" / "images"
orig_val_labels_dir = dataset_root / "val" / "labels"
test_images_dir = dataset_root / "test" / "images"

In [5]:
for p in (train_images_dir, orig_train_labels_dir, val_images_dir, orig_val_labels_dir, test_images_dir):
    if not p.exists():
        print(f"Warning: expected path not found: {p}")

In [6]:
wrk = Path("/kaggle/working/cotton_dataset")
wrk_train_labels = wrk / "train" / "labels"
wrk_val_labels = wrk / "val" / "labels"
os.makedirs(wrk_train_labels, exist_ok=True)
os.makedirs(wrk_val_labels, exist_ok=True)

def copy_label_files(src_dir: Path, dst_dir: Path):
    txts = sorted(glob.glob(str(src_dir / "*.txt"))) if src_dir.exists() else []
    for t in txts:
        dst = dst_dir / Path(t).name
        if not dst.exists():
            shutil.copy2(t, str(dst))

copy_label_files(orig_train_labels_dir, wrk_train_labels)
copy_label_files(orig_val_labels_dir, wrk_val_labels)

train_labels_dir = wrk_train_labels
val_labels_dir = wrk_val_labels

print("Dataset directories:")
print(" train_images_dir ->", train_images_dir)
print(" train_labels_dir ->", train_labels_dir)
print(" val_images_dir   ->", val_images_dir)
print(" val_labels_dir   ->", val_labels_dir)
print(" test_images_dir  ->", test_images_dir)

Dataset directories:
 train_images_dir -> /kaggle/input/the-3lc-cotton-weed-detection-challenge/cotton_weed_competition_dataset/train/images
 train_labels_dir -> /kaggle/working/cotton_dataset/train/labels
 val_images_dir   -> /kaggle/input/the-3lc-cotton-weed-detection-challenge/cotton_weed_competition_dataset/val/images
 val_labels_dir   -> /kaggle/working/cotton_dataset/val/labels
 test_images_dir  -> /kaggle/input/the-3lc-cotton-weed-detection-challenge/cotton_weed_competition_dataset/test/images


In [7]:
def read_yolo_label(path):
    boxes = []
    if not os.path.exists(path):
        return boxes
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 5:
                continue
            cls = int(parts[0])
            vals = list(map(float, parts[1:5]))
            boxes.append((cls, *vals))
    return boxes

def write_yolo_label(path, boxes):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        for b in boxes:
            f.write("{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(int(b[0]), b[1], b[2], b[3], b[4]))

In [8]:
def yolo_to_xyxy(box, img_w, img_h):
    x_c, y_c, w, h = box
    x1 = (x_c - w/2) * img_w
    y1 = (y_c - h/2) * img_h
    x2 = (x_c + w/2) * img_w
    y2 = (y_c + h/2) * img_h
    return [x1, y1, x2, y2]

def xyxy_to_yolo(x1, y1, x2, y2, img_w, img_h):
    w = max(0, x2 - x1)
    h = max(0, y2 - y1)
    x_c = x1 + w/2
    y_c = y1 + h/2
    if img_w == 0 or img_h == 0:
        return [0.5, 0.5, 0.0, 0.0]
    return [x_c/img_w, y_c/img_h, w/img_w, h/img_h]

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    inter_w = max(0, inter_x2 - inter_x1)
    inter_h = max(0, inter_y2 - inter_y1)
    inter_area = inter_w * inter_h
    area_a = max(0, ax2-ax1) * max(0, ay2-ay1)
    area_b = max(0, bx2-bx1) * max(0, by2-by1)
    union = area_a + area_b - inter_area
    if union == 0:
        return 0.0
    return inter_area / union

In [9]:
MIN_REL_AREA = 0.0008 

def sanitize_labels(images_dir, labels_dir, min_rel_area=MIN_REL_AREA, dry_run=False):
    print("Sanitizing labels in:", labels_dir)
    modified = 0
    total_boxes = 0
    label_paths = sorted(glob.glob(str(labels_dir / "*.txt")))
    for lab in label_paths:
        img_name = Path(lab).stem
        img_path = None
        for ext in ("jpg","jpeg","png","JPG","PNG"):
            cand = images_dir / (img_name + "." + ext)
            if cand.exists():
                img_path = cand
                break
        if img_path is None:
            continue
        img = Image.open(img_path)
        w,h = img.size
        boxes = read_yolo_label(lab)
        total_boxes += len(boxes)
        new_boxes = []
        for b in boxes:
            cls, x_c, y_c, bw, bh = b
            x_c = min(0.999999, max(0.0, x_c))
            y_c = min(0.999999, max(0.0, y_c))
            bw = min(0.999999, max(0.0, bw))
            bh = min(0.999999, max(0.0, bh))
            area_rel = bw * bh
            if area_rel < min_rel_area:
                continue
            if bw <= 0 or bh <= 0:
                continue
            new_boxes.append((cls, x_c, y_c, bw, bh))
        kept = []
        for cand in sorted(new_boxes, key=lambda x: x[3]*x[4], reverse=True):
            cls, x_c, y_c, bw, bh = cand
            cand_xy = yolo_to_xyxy((x_c,y_c,bw,bh), w, h)
            overlap = False
            for k in kept:
                if k[0] != cls:
                    continue
                k_xy = yolo_to_xyxy((k[1],k[2],k[3],k[4]), w, h)
                if iou_xyxy(cand_xy, k_xy) > 0.85:
                    overlap = True
                    break
            if not overlap:
                kept.append(cand)
        if dry_run:
            if len(kept) != len(boxes):
                modified += 1
        else:
            write_yolo_label(lab, kept)
            if len(kept) != len(boxes):
                modified += 1
    print(f"Sanitation complete. Modified files: {modified}. Total input boxes: {total_boxes}")
    return modified
sanitize_labels(train_images_dir, train_labels_dir, dry_run=False)
sanitize_labels(val_images_dir, val_labels_dir, dry_run=False)

Sanitizing labels in: /kaggle/working/cotton_dataset/train/labels
Sanitation complete. Modified files: 7. Total input boxes: 969
Sanitizing labels in: /kaggle/working/cotton_dataset/val/labels
Sanitation complete. Modified files: 3. Total input boxes: 269


3

In [10]:
data_yaml_path = Path("cotton_yolo_data.yaml")
data_yaml = {
    "names": {0: "carpetweed", 1: "morningglory", 2: "palmeramaranth"},
    "nc": 3,
    "train": str(train_images_dir.resolve()),
    "val": str(val_images_dir.resolve()),
    "test": str(test_images_dir.resolve())
}
import yaml
with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)
print("Wrote data yaml to", data_yaml_path)

Wrote data yaml to cotton_yolo_data.yaml


In [11]:
import torch

device = 0 if torch.cuda.is_available() else "cpu"
gpu = torch.cuda.is_available()

imgsz = 896             
bs = 8 if gpu else 4    

epochs_baseline = 100

MIN_REL_AREA = 0.0005  

CONF_THR = 0.70
IOU_MATCH = 0.50

FINAL_CONF_THR = 0.25

train_args = dict(
    data=str(data_yaml_path),
    epochs=epochs_baseline,
    imgsz=imgsz,
    batch=bs,
    device=device,
    name="yolov8n_baseline",
    exist_ok=True,
    workers=8,          
    mosaic=1,            
    mixup=0.20,         
    lr0=0.0015,           
    weight_decay=1e-4,  
    degrees=15.0,
    translate=0.20,
    scale=0.25,
    shear=2.5,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.45,
    cache=True,         
    patience=50         
)

retrain_epochs = 60
retrain_name = "yolov8n_selftrain"
retrain_args = dict(
    data=str(data_yaml_path),
    epochs=retrain_epochs,
    imgsz=imgsz,
    batch=bs,
    device=device,
    name=retrain_name,
    exist_ok=True,
    workers=8,
    lr0=0.0010,          
    weight_decay=1e-4,
    mosaic=1,
    mixup=0.15,
    degrees=12.0,
    translate=0.12,
    scale=0.15,
    shear=2.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.45,
    cache=True,
    patience=40
)


In [12]:
print("Initializing YOLOv8n model (pretrained COCO).")
model = YOLO("yolov8n.pt")  

print("Starting baseline training (this step may be time-consuming).")
try:
    results = model.train(**train_args)
except Exception as e:
    print("Error during baseline training. Exception follows:")
    raise

runs_dir = Path("runs") / "detect" / train_args["name"]
best_pt = None
if runs_dir.exists():
    ckpts = sorted(runs_dir.glob("weights/*.pt"), key=os.path.getmtime)
    if ckpts:
        best_pt = str(ckpts[-1])
print("Baseline best checkpoint:", best_pt or "Not found; model object will be used.")

Initializing YOLOv8n model (pretrained COCO).
Starting baseline training (this step may be time-consuming).
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cotton_yolo_data.yaml, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.45, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1, multi_scale=False, name=yolov8n_baseline, nbs=64, nms=False, opse

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        133        269      0.881      0.726      0.831      0.581
            carpetweed         44        110      0.852      0.609      0.775      0.488
          morningglory         54         70      0.951      0.829      0.913      0.687
        palmeramaranth         51         89       0.84      0.742      0.804      0.568
Speed: 0.3ms preprocess, 2.9ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to /kaggle/working/runs/detect/yolov8n_baseline
Baseline best checkpoint: runs/detect/yolov8n_baseline/weights/best.pt


In [13]:
ONF_THR = 0.70         
IOU_MATCH = 0.50        
MIN_REL_AREA = 0.0005   

trained_model = model  

train_img_paths = sorted(glob.glob(str(train_images_dir / "*.*")))
train_label_paths = {Path(p).stem: p for p in glob.glob(str(train_labels_dir / "*.txt"))}

# helpers to safely extract scalar and xyxy array
def tensor_to_float(t):
    try:
        return float(t.item())
    except Exception:
        return float(np.array(t).squeeze())

def xyxy_tensor_to_list(box_xyxy):
    # box_xyxy may be a tensor of shape (1,4) or (4,)
    arr = np.array(box_xyxy.cpu())
    arr = np.asarray(arr).reshape(-1)
    if arr.size < 4:
        # fallback to zeros (should not happen)
        return [0.0, 0.0, 0.0, 0.0]
    return [float(arr[0]), float(arr[1]), float(arr[2]), float(arr[3])]

def mine_pseudo_labels(trained_model, img_paths, labels_dir, conf_thr=CONF_THR, iou_match=IOU_MATCH, min_rel_area=MIN_REL_AREA):
    added = 0
    explored = 0
    for img_p in img_paths:
        stem = Path(img_p).stem
        explored += 1

        res = trained_model.predict(source=img_p, imgsz=imgsz, conf=conf_thr, iou=0.45, device=device, verbose=False)
        if not res:
            continue
        r = res[0]
        # r.boxes may be an object with fields; if no boxes, skip
        if not hasattr(r, "boxes") or len(r.boxes) == 0:
            continue

        # image size
        img = Image.open(img_p)
        w, h = img.size

        preds = []
        for box in r.boxes:
            conf = tensor_to_float(box.conf)
            cls = int(tensor_to_float(box.cls))
            xyxy_list = xyxy_tensor_to_list(box.xyxy)
            x1, y1, x2, y2 = xyxy_list
            yolo_box = xyxy_to_yolo(x1, y1, x2, y2, w, h)
            rel_area = float(yolo_box[2]) * float(yolo_box[3])
            if conf < conf_thr or rel_area < min_rel_area:
                continue
            preds.append((cls, conf, yolo_box))

        if not preds:
            continue

        gt_path = os.path.join(labels_dir, stem + ".txt")
        gt_boxes = read_yolo_label(gt_path)
        gt_xy = []
        for g in gt_boxes:
            _, x_c, y_c, bw, bh = g
            gt_xy.append(yolo_to_xyxy((x_c, y_c, bw, bh), w, h))

        to_add = []
        for cls, conf, ybox in preds:
            pred_xy = yolo_to_xyxy((ybox[0], ybox[1], ybox[2], ybox[3]), w, h)
            overlaps = [iou_xyxy(pred_xy, gxy) for gxy in gt_xy] if gt_xy else []
            max_iou = max(overlaps) if overlaps else 0.0
            # only add if it does not match existing GT (likely missed annotation)
            if max_iou < iou_match:
                to_add.append((cls, ybox))

        if to_add:
            # append to writable label file
            label_file = os.path.join(labels_dir, stem + ".txt")
            existing = read_yolo_label(label_file)
            for cls, yb in to_add:
                existing.append((cls, float(yb[0]), float(yb[1]), float(yb[2]), float(yb[3])))
                added += 1
            write_yolo_label(label_file, existing)

    print(f"Pseudo-label mining complete. Images scanned: {explored}. Boxes added: {added}")
    return added

print("Starting pseudo-label mining (high-confidence training set predictions appended to label files).")
added = mine_pseudo_labels(trained_model, train_img_paths, str(train_labels_dir))

Starting pseudo-label mining (high-confidence training set predictions appended to label files).
Pseudo-label mining complete. Images scanned: 542. Boxes added: 15


In [14]:
retrain_epochs = 60
retrain_name = "yolov8n_selftrain"
retrain_args = dict(
    data=str(data_yaml_path),
    epochs=retrain_epochs,
    imgsz=imgsz,
    batch=bs,
    device=device,
    name=retrain_name,
    exist_ok=True,
    workers=8,
    lr0=0.0015,
    weight_decay=1e-4,
    mosaic=1,
    mixup=0.15,
    degrees=12.0,
    translate=0.12,
    scale=0.15,
    shear=2.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.45
)
baseline_run_dir = Path("runs") / "detect" / train_args["name"]
best_ckpt = None
if baseline_run_dir.exists():
    # prefer 'weights/best.pt', fallback to 'weights/last.pt'
    cand_best = baseline_run_dir / "weights" / "best.pt"
    cand_last = baseline_run_dir / "weights" / "last.pt"
    if cand_best.exists():
        best_ckpt = str(cand_best)
    elif cand_last.exists():
        best_ckpt = str(cand_last)

if best_ckpt is None:
    print("Warning: baseline checkpoint not found; retraining will start from pretrained yolov8n weights.")
    best_ckpt = "yolov8n.pt"
print("Retrain: using checkpoint:", best_ckpt)
model_for_retrain = YOLO(best_ckpt)

print("Starting retraining after pseudo-label augmentation.")
try:
    retrain_results = model_for_retrain.train(**retrain_args)
    trained_model = model_for_retrain
except Exception as e:
    print("Error during retraining. Exception follows:")
    raise

Retrain: using checkpoint: runs/detect/yolov8n_baseline/weights/best.pt
Starting retraining after pseudo-label augmentation.
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cotton_yolo_data.yaml, degrees=12.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.45, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=runs/detect/yolov8n_baseline/weights/best.pt, momentum=0.937, mosaic=1, multi_scale=F

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        133        269      0.802      0.773      0.821       0.59
            carpetweed         44        110      0.754      0.698      0.737      0.498
          morningglory         54         70       0.91      0.857      0.892      0.664
        palmeramaranth         51         89      0.742      0.764      0.833      0.608
Speed: 0.5ms preprocess, 4.4ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to /kaggle/working/runs/detect/yolov8n_selftrain


In [15]:
test_image_paths = sorted(glob.glob(str(test_images_dir / "*.*")))
sub_rows = []
FINAL_CONF_THR = 0.25 

def infer_with_tta(model, img_path, imgsz=imgsz, conf=0.001, iou=0.45, tta=True):
    detections = []

    def parse_result(res_obj, img_w, img_h):
        parsed = []
        if not res_obj or not hasattr(res_obj, "boxes") or len(res_obj.boxes) == 0:
            return parsed
        for box in res_obj.boxes:
            conf = tensor_to_float(box.conf)
            cls = int(tensor_to_float(box.cls))
            xyxy_list = xyxy_tensor_to_list(box.xyxy)
            x1,y1,x2,y2 = xyxy_list
            ybox = xyxy_to_yolo(x1,y1,x2,y2, img_w, img_h)
            parsed.append((cls, conf, ybox))
        return parsed

    base_res = model.predict(source=img_path, imgsz=imgsz, conf=conf, iou=iou, device=device, verbose=False)
    if base_res:
        # determine image size once
        img = Image.open(img_path)
        w,h = img.size
        detections += parse_result(base_res[0], w, h)

    if tta:
        img = Image.open(img_path)
        w,h = img.size
        img_flipped = ImageOps.mirror(img)
        tmp_path = "/tmp/tta_tmp.jpg"
        img_flipped.save(tmp_path)
        res_f = model.predict(source=tmp_path, imgsz=imgsz, conf=conf, iou=iou, device=device, verbose=False)
        if res_f:
            parsed = []
            for box in res_f[0].boxes if hasattr(res_f[0], "boxes") else []:
                c = tensor_to_float(box.conf)
                cl = int(tensor_to_float(box.cls))
                xyxy_list = xyxy_tensor_to_list(box.xyxy)
                x1f,y1f,x2f,y2f = xyxy_list
                mx1 = w - x2f
                mx2 = w - x1f
                ybox = xyxy_to_yolo(mx1, y1f, mx2, y2f, w, h)
                parsed.append((cl, c, ybox))
            detections += parsed

    final = []
    detections_sorted = sorted(detections, key=lambda x: x[1], reverse=True)
    img_w, img_h = Image.open(img_path).size
    for cls, conf, ybox in detections_sorted:
        keep = True
        xy = yolo_to_xyxy((ybox[0], ybox[1], ybox[2], ybox[3]), img_w, img_h)
        for k in final:
            if k[0] != cls:
                continue
            kxy = yolo_to_xyxy((k[2][0], k[2][1], k[2][2], k[2][3]), img_w, img_h)
            if iou_xyxy(xy, kxy) > 0.45:
                keep = False
                break
        if keep:
            final.append((cls, conf, ybox))
    return final

print("Running inference on test images (this will take time).")
for timg in test_image_paths:
    stem = Path(timg).stem
    dets = infer_with_tta(trained_model, timg, imgsz=imgsz, conf=0.001, iou=0.45, tta=True)
    # format prediction string
    parts = []
    for cls, conf, ybox in dets:
        if conf < FINAL_CONF_THR:
            continue
        parts += [str(int(cls)), f"{float(conf):.3f}", f"{ybox[0]:.6f}", f"{ybox[1]:.6f}", f"{ybox[2]:.6f}", f"{ybox[3]:.6f}"]
    pred_str = " ".join(parts) if parts else "no box"
    sub_rows.append({"image_id": stem, "prediction_string": pred_str})

submission_df = pd.DataFrame(sub_rows)
all_test_stems = [Path(p).stem for p in test_image_paths]
existing_stems = set(submission_df['image_id'].tolist())
for s in all_test_stems:
    if s not in existing_stems:
        submission_df = submission_df.append({"image_id": s, "prediction_string": "no box"}, ignore_index=True)

submission_df = submission_df[["image_id","prediction_string"]]
submission_csv = "submission.csv"
submission_df.to_csv(submission_csv, index=False)
print("Submission saved to", submission_csv)
print(submission_df.head(10))

Running inference on test images (this will take time).
Submission saved to submission.csv
                   image_id                                  prediction_string
0      20190613_6062W_CM_29        1 0.887 0.490217 0.490678 0.413329 0.773381
1      20190613_6062W_CM_36  2 0.890 0.548463 0.656367 0.875511 0.619929 2 ...
2      20190613_6062W_CM_57  2 0.811 0.565868 0.954940 0.236748 0.088851 1 ...
3  20190613_Pixel3aXL_CM_65                                             no box
4    20190718_iPhone7_EB_12        1 0.401 0.142776 0.129898 0.284856 0.258897
5    20200526_iPhone8_CH_15  2 0.775 0.575500 0.585648 0.848999 0.828704 2 ...
6    20200526_iPhone8_CH_17  1 0.601 0.597162 0.340035 0.805676 0.671811 1 ...
7       20200612_iPhone6_11        0 0.855 0.237689 0.333865 0.321492 0.360767
8      20200612_iPhone6_B_2        1 0.902 0.488331 0.407231 0.537457 0.436965
9   20200612_iPhone6_SY_152        0 0.855 0.237689 0.333865 0.321492 0.360767
